---
title: "Trade Envy Report"
date: last-modified
engine: jupyter
execute:
  echo: false
  enabled: true
---

In [2]:
from cow_amm_trade_envy.datasources import get_logs
from cow_amm_trade_envy.constants import DB_FILE
import plotly.graph_objects as go
import duckdb
from cow_amm_trade_envy.models import Pools


def analyze_surplus(network):
    table_name = f"{network}_envy"
    with duckdb.connect(database=DB_FILE) as conn:
        df = conn.execute(f"SELECT * FROM {table_name}").fetchdf()

    def isin_pool(topic: str, pool_address: str) -> bool:
        if pool_address == "None":
            return False
        assert "0x" == pool_address[:2]
        return pool_address[2:] in topic

    def log_is_used(log, pool_address: str) -> bool:
        return any(isin_pool(topic, pool_address) for topic in log["topics"])

    def logs_are_used(logs, pool_address: str) -> bool:
        return any(log_is_used(log, pool_address) for log in logs)

    df["logs"] = df["call_tx_hash"].apply(get_logs)
    # df has logs and pool cols
    df["is_used"] = df.apply(
        lambda row: logs_are_used(row["logs"], row["pool"]), axis=1
    )

    df = df[df["trade_envy"] > 0]

    # Group by pool and calculate unused and used surplus
    grouped = df.groupby("pool").apply(
        lambda g: {
            "unused_surplus": g[~g["is_used"]]["trade_envy"].sum(),
            "used_surplus": g[g["is_used"]]["trade_envy"].sum(),
        }, include_groups=False
    )

    results = grouped.reset_index().rename(columns={0: "surplus"})
    results["unused_surplus"] = results["surplus"].apply(lambda x: x["unused_surplus"])
    results["used_surplus"] = results["surplus"].apply(lambda x: x["used_surplus"])
    results["pool"] = results["pool"].apply(lambda x: Pools().get_name_from_address(x))
    return results


network = "ethereum"
surplus_data = analyze_surplus(network)

# Prepare data for the stacked bar plot
pools = surplus_data["pool"]
unused_surplus = surplus_data["unused_surplus"]
used_surplus = surplus_data["used_surplus"]

fig = go.Figure()

# Add unused surplus (bottom of the stack)
fig.add_trace(
    go.Bar(
        name="Unused Surplus",
        x=pools,
        y=unused_surplus,
        marker_color="green",
    )
)

# Add used surplus (top of the stack)
fig.add_trace(
    go.Bar(
        name="Used Surplus",
        x=pools,
        y=used_surplus,
        marker_color="orange",
    )
)

# Update layout for better visualization
fig.update_layout(
    title="Comparison of Surplus Usage by Pool",
    xaxis_title="Pools",
    yaxis_title="Surplus Amount",
    template="plotly",
    barmode="stack",  # Make the bars stacked
)

# Show the plot
fig.show()
